# Model Prep.
### There are two ways to get the model:

1. Make it form scratch and don't froget to convert it to proto buffer (pb) fromat to use it in OpenCV as config file
2. use a pre-trained one to work with.


- I used Tensorflow pre-trained models and already prepared config pb from here: https://github.com/opencv/opencv/wiki/TensorFlow-Object-Detection-API you can find a tutorial of how to make a config pb file.
- I choose Mobilenet v1 which is trained on cocco dataset you can find such info from here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md

- you can find the file of cocco labels from here: https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_label_map.pbtxt


In [4]:
file = open('mscoco_label_map.pbtxt.txt')
lines =file.read()
lines.split('\nitem')[:3]

['item {\n  name: "/m/01g317"\n  id: 1\n  display_name: "person"\n}',
 ' {\n  name: "/m/0199g"\n  id: 2\n  display_name: "bicycle"\n}',
 ' {\n  name: "/m/0k4j"\n  id: 3\n  display_name: "car"\n}']

In [1]:
#prepare labels
file = open('mscoco_label_map.pbtxt.txt')
labels = ["null"] #because the first label with an id 1 in the label_map.pbtxt
txt =file.read()
line = txt.split('\nitem')
ind= [i.split('display_name: "') for i in line]
for i in range(len(ind)):
    labels.append(ind[i][1].replace('"\n}',''))
labels[:3]    

['null', 'person', 'bicycle']

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
model_path = 'ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb'
config_file = 'ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt'
model = cv2.dnn.readNetFromTensorflow(model_path,config_file)
cap = cv2.VideoCapture(0)
while (True):
    success, frame = cap.read()
    base_img = frame.copy()
    original_size = frame.shape
    tartget_size = (300,300)
    img = cv2.resize(frame,tartget_size)
    aspect_ratio_y = original_size[0] / tartget_size [0] #help us in to apply the work on base_img
    aspect_ratio_x = original_size[1] / tartget_size[1]
    imgblob= cv2.dnn.blobFromImage(img) #prepare img and returen 4-dim (1,3,300,300)
    model.setInput(imgblob)
    detections = model.forward()
    detections_df = pd.DataFrame(detections[0][0], columns = ["img_id", "label", "confidence", "left", "top", "right", "bottom"])
    detections_df = detections_df.query('confidence > 0.9') #label of 1 means face\ 0:means background
    for i,instance in detections_df.iterrows():
        text = labels[int(detections_df['label'][i])] + str(round(detections_df['confidence'][i]*100)) + "%"
        right = int(instance['right'] * 300) # multiply by 300 to see the results on the base_img remeber because we resize the img
        left = int(instance['left'] * 300)
        top = int(instance['top'] * 300)
        bottom = int(instance ['bottom'] * 300)
        detect_face = base_img[int(top *aspect_ratio_y): int(bottom*aspect_ratio_y), int(left*aspect_ratio_x): int(right*aspect_ratio_x)]
        if detect_face.shape[0] > 0 and detect_face.shape[1] > 0:
            cv2.putText(base_img, text , (int(left*aspect_ratio_x), int(top*aspect_ratio_y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 200), 2)
            cv2.rectangle(base_img, (int(left*aspect_ratio_x), int(top*aspect_ratio_y)), (int(right*aspect_ratio_x), int(bottom*aspect_ratio_y)), (255, 0, 0), 4) #draw rectangle to main image
    cv2.imshow('Object Detection', base_img) #show window of your camera
    if cv2.waitKey(1) & 0xFF == ord('q'): #press q To exit
        break
    
    
cap.release()    
cv2.destroyAllWindows()
